In [1]:
from pyspark.sql import functions as F
# scan_df = df_scan
# rrm_radar_df = df_radar
from datetime import datetime

env = "staging"
env = "production"

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)
date_day = "2020-06-*"
hr = "*" #now().hour-1

rrm_radar_path = s3_bucket + "rrm_radar_channel_stat/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)
print(rrm_radar_path)


date_day = datetime.today().strftime('%Y-%m-%d') #"2020-05-*"
curr_hr = '*'  # latest 
ap_neighbors_path = s3_bucket + "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

print(ap_neighbors_path)


scan_df = spark.read.csv(ap_neighbors_path, header=True)                   
rrm_radar_df = spark.read.csv(rrm_radar_path, header=True)    

scan_df.show(2)
rrm_radar_df.show(2)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
27,application_1591015534858_0031,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
s3://mist-aggregated-stats-production/aggregated-stats/rrm_radar_channel_stat/dt=2020-06-*/hr=*/*.csv
s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-07-10/hr=*/*.csv
+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|0000063a-a4ce-409...|d420b000ee6b|d420b000ee61|  24|1594342685|-71.0|
|0000063a-a4ce-409...|d420b000ee7f|d420b000ee75|  24|1594342413|-87.0|
+--------------------+------------+------------+----+----------+-----+
only showing top 2 rows

+--------------------+------------+-----------+-------------------+---------------+----------+
|             site_id|       ap_id|pre_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-----------+-------------------+---------------+----------+
|ad1fe7ce-

In [2]:
rrm_radar_df = rrm_radar_df.withColumnRenamed("pre_channel", "radar_channel")
rrm_radar_df.show(3)

+--------------------+------------+-------------+-------------------+---------------+----------+
|             site_id|       ap_id|radar_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-------------+-------------------+---------------+----------+
|ad1fe7ce-b78d-458...|5c5b352f5707|          132|                  1|     1592678555|1592676000|
|1cebf4b4-3b20-4db...|5c5b35bf2fd9|           60|                  1|     1592679386|1592676000|
|13d68843-7ebd-42a...|d420b0008b8d|           52|                  1|     1592678522|1592676000|
+--------------------+------------+-------------+-------------------+---------------+----------+
only showing top 3 rows

In [3]:
rrm_radar_df.printSchema()

root
 |-- site_id: string (nullable = true)
 |-- ap_id: string (nullable = true)
 |-- radar_channel: string (nullable = true)
 |-- radar_channel_count: string (nullable = true)
 |-- last_radar_time: string (nullable = true)
 |-- time: string (nullable = true)

# Radar event counts by Sites

In [4]:
radar_by_site_df = rrm_radar_df.select('site_id')\
                                    .groupBy('site_id')\
                                    .agg(F.count('site_id').alias('site_counts'))
radar_by_site_df.show(10)

+--------------------+-----------+
|             site_id|site_counts|
+--------------------+-----------+
|9d81b9a7-93e7-426...|        177|
|abb86cde-14ab-432...|        101|
|9c85b58d-d433-476...|         18|
|cb777830-5c74-430...|          8|
|6805fa28-f845-405...|          1|
|c51e612c-4139-4ef...|          7|
|f420dbd2-559e-453...|          5|
|69b07bf8-a809-427...|          1|
|2c740d9b-89d5-45a...|          3|
|5ff25c03-2b1f-4b9...|          1|
+--------------------+-----------+
only showing top 10 rows

In [5]:
radar_by_site_df= radar_by_site_df.orderBy(radar_by_site_df["site_counts"].desc())
radar_by_site_df.show(20, False)

+------------------------------------+-----------+
|site_id                             |site_counts|
+------------------------------------+-----------+
|a3edc895-aae6-4e40-8ee0-0f7198bdcc0c|2838       |
|934d5efe-5e35-48dd-99f9-5c57534708aa|1517       |
|866808ba-1ff2-4e99-8e17-34ee1d3c70f4|1418       |
|c16dcb76-15ef-43b1-b531-ffe9f52e119e|1140       |
|332b319a-aebd-4820-b2b4-645c247cdd79|700        |
|0112b19a-f4d4-4f25-ac8b-a2516bb96424|551        |
|3cf29ebc-21f1-49e5-9956-25495fe8f41b|487        |
|801109d5-f7b3-4818-9447-bfcc221abc2a|437        |
|b3c32004-a0dc-4353-b697-1065fbd4f639|416        |
|13d68843-7ebd-42a1-8528-495c41ccf8ef|369        |
|874a8612-bf55-470d-8edd-e610df55db8e|358        |
|d9ea1152-f455-442d-9bb9-5e39906ce0da|318        |
|5fe533f6-39f5-431f-90eb-c9083a962e7d|294        |
|38b8e0f6-0a87-4c15-909e-1a1d13a395ea|284        |
|1b480105-0ae1-41cb-9a81-f28fbbe1d3c6|255        |
|5b3926db-e1cc-4ff4-a115-d3271589ab8a|223        |
|67c41b47-e8a1-4495-8b96-15c2d3

In [6]:
radar_impacted_sites = [ x.site_id for x in radar_by_site_df.select('site_id').limit(10).collect()]

In [7]:
import requests
papi_site = "http://papi-{env}.mistsys.net/internal/sites/".format(env=env)
papi_org = "http://papi-{env}.mistsys.net/internal/orgs/".format(env=env)
for site in radar_impacted_sites:
#     print(site)
    papi_site_id = papi_site+ site
    res= requests.get(papi_site_id)
    res = res.json()
    org_id = res.get("org_id")
    papi_org_id = papi_org+ org_id
    res_org= requests.get(papi_org_id)
    res_org = res_org.json()
    
    print(site, res_org.get("name"), res.get("name"))
 

a3edc895-aae6-4e40-8ee0-0f7198bdcc0c UDoN グラントウキョウサウスタワー
934d5efe-5e35-48dd-99f9-5c57534708aa Indeed nrtoff17
866808ba-1ff2-4e99-8e17-34ee1d3c70f4 Jumeirah BAA
c16dcb76-15ef-43b1-b531-ffe9f52e119e Wild Wadi Wild Wadi
332b319a-aebd-4820-b2b4-645c247cdd79 Juniper Networks ME-UA-DUB
0112b19a-f4d4-4f25-ac8b-a2516bb96424 AmazonProd DBN2
3cf29ebc-21f1-49e5-9956-25495fe8f41b SYSTEMTECH DUBAI SOUTH
801109d5-f7b3-4818-9447-bfcc221abc2a Group 1 Automotive TX-Houston-Ford
b3c32004-a0dc-4353-b697-1065fbd4f639 SPACEWAY NPCC
13d68843-7ebd-42a1-8528-495c41ccf8ef WeWork SFO10

In [8]:
# res.json()

site_1 = "13d68843-7ebd-42a1-8528-495c41ccf8ef"


# Radar_last_seen_in_site

In [9]:
# last seen in site

last_seen_in_site_df = rrm_radar_df.select('site_id', 'radar_channel', 'last_radar_time')\
                                    .groupBy('site_id', 'radar_channel')\
                                    .agg(F.max('last_radar_time').alias('last_seen_in_site'))
last_seen_in_site_df.show(10)

+--------------------+-------------+-----------------+
|             site_id|radar_channel|last_seen_in_site|
+--------------------+-------------+-----------------+
|053bfd85-f7bd-45e...|          134|       1591136988|
|0f6f1b85-a702-485...|          104|       1592995648|
|11e4c4dd-bc3e-4b1...|          132|       1593117128|
|1321bc06-afdf-479...|          132|       1592286507|
|1d42aca2-278a-43b...|           60|       1593068460|
|1f68eb07-89e9-4c3...|           56|       1591066914|
|227e59a8-23c7-4b0...|          124|       1592853954|
|325e444a-10e2-4db...|           64|       1591795529|
|36d8d839-7e98-491...|           52|       1592679636|
|374c790a-a979-429...|          108|       1593154433|
+--------------------+-------------+-----------------+
only showing top 10 rows

In [10]:
# AP's primary hit


In [11]:
rrm_radar_primary_df = rrm_radar_df\
                                .groupBy('site_id', 'ap_id', 'radar_channel')\
                                .agg(F.sum('radar_channel_count').alias('radar_channel_count'),
                                 F.max('last_radar_time').alias('last_seen'))
rrm_radar_primary_df.show(10)

+--------------------+------------+-------------+-------------------+----------+
|             site_id|       ap_id|radar_channel|radar_channel_count| last_seen|
+--------------------+------------+-------------+-------------------+----------+
|00272816-8dc2-4f1...|5c5b35ae6e05|          140|                1.0|1592513938|
|03ba030e-2a1f-4d5...|5c5b359ed985|          140|                1.0|1593228635|
|0b367ea0-0443-417...|5c5b3551a25c|          124|                1.0|1592925185|
|13d68843-7ebd-42a...|5c5b35bf2296|          136|                1.0|1592678532|
|191f33b1-2271-420...|d420b08102d9|          128|                1.0|1592030622|
|1b480105-0ae1-41c...|5c5b355063f6|          128|                2.0|1592380039|
|1cebf4b4-3b20-4db...|5c5b35bf2fd9|           60|                1.0|1592679386|
|1f484765-a3d2-466...|d420b00101df|           52|                1.0|1591685973|
|216d8f81-b5a4-483...|5c5b35538b71|          116|                8.0|1591682457|
|2550a4b8-f77b-457...|5c5b35

In [12]:
rrm_radar_primary_df = rrm_radar_df\
                                .groupBy('site_id', 'ap_id', 'radar_channel')\
                                .agg(F.sum('radar_channel_count').alias('radar_channel_count'),
                                 F.max('last_radar_time').alias('last_seen'))
rrm_radar_primary_df.show(10)

+--------------------+------------+-------------+-------------------+----------+
|             site_id|       ap_id|radar_channel|radar_channel_count| last_seen|
+--------------------+------------+-------------+-------------------+----------+
|00272816-8dc2-4f1...|5c5b35ae6e05|          140|                1.0|1592513938|
|03ba030e-2a1f-4d5...|5c5b359ed985|          140|                1.0|1593228635|
|0b367ea0-0443-417...|5c5b3551a25c|          124|                1.0|1592925185|
|13d68843-7ebd-42a...|5c5b35bf2296|          136|                1.0|1592678532|
|191f33b1-2271-420...|d420b08102d9|          128|                1.0|1592030622|
|1b480105-0ae1-41c...|5c5b355063f6|          128|                2.0|1592380039|
|1cebf4b4-3b20-4db...|5c5b35bf2fd9|           60|                1.0|1592679386|
|1f484765-a3d2-466...|d420b00101df|           52|                1.0|1591685973|
|216d8f81-b5a4-483...|5c5b35538b71|          116|                8.0|1591682457|
|2550a4b8-f77b-457...|5c5b35

In [13]:
#for select site
rrm_radar_primary_df.filter( col("site_id") == site_1).show()
# rrm_radar_primary_df.show()

name 'col' is not defined
Traceback (most recent call last):
NameError: name 'col' is not defined



# AP's neighbor impact

In [14]:
#  ap-ap2 neighbor relationship

scan_agg_df=  scan_df.groupBy('site', 'ap', 'ap2').agg(F.max('rssi').alias("max_rssi"),
                                                     F.count('ap2').alias('scan_counts')) 
scan_agg_df.show(2) 

+--------------------+------------+------------+--------+-----------+
|                site|          ap|         ap2|max_rssi|scan_counts|
+--------------------+------------+------------+--------+-----------+
|0000063a-a4ce-409...|d420b000ee7f|d420b000f0be|   -83.0|          4|
|0000063a-a4ce-409...|d420b080eb28|d420b080eb69|   -72.0|          8|
+--------------------+------------+------------+--------+-----------+
only showing top 2 rows

In [15]:
# inner-join with rrm-radar_df
ap_neighbor_radar_2 = scan_agg_df\
         .filter('ap2 is not Null and max_rssi >-75')\
         .join(rrm_radar_primary_df, [scan_agg_df.site == rrm_radar_primary_df.site_id,
                                      scan_agg_df.ap2 == rrm_radar_df.ap_id], 'inner')

ap_neighbor_radar_2.show(5)

+--------------------+------------+------------+--------+-----------+--------------------+------------+-------------+-------------------+----------+
|                site|          ap|         ap2|max_rssi|scan_counts|             site_id|       ap_id|radar_channel|radar_channel_count| last_seen|
+--------------------+------------+------------+--------+-----------+--------------------+------------+-------------+-------------------+----------+
|0009dab3-7a6b-4ac...|5c5b358e2878|5c5b358e297c|   -57.0|          8|0009dab3-7a6b-4ac...|5c5b358e297c|           52|                1.0|1592430500|
|00544ad7-df28-4b9...|5c5b358ef9b5|5c5b358f1620|   -65.0|          8|00544ad7-df28-4b9...|5c5b358f1620|           52|                1.0|1592334948|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|   -56.0|          8|0112b19a-f4d4-4f2...|5c5b352ec92c|          112|                3.0|1591074919|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|   -56.0|          8|0112b19a-f4d4-4f2...|5c5b352ec92c|   

In [16]:
ap_neighbor_radar_2.filter('radar_channel>0').show(5)
# ap_neighbor_radar_2.show()

+--------------------+------------+------------+--------+-----------+--------------------+------------+-------------+-------------------+----------+
|                site|          ap|         ap2|max_rssi|scan_counts|             site_id|       ap_id|radar_channel|radar_channel_count| last_seen|
+--------------------+------------+------------+--------+-----------+--------------------+------------+-------------+-------------------+----------+
|0009dab3-7a6b-4ac...|5c5b358e2878|5c5b358e297c|   -57.0|          8|0009dab3-7a6b-4ac...|5c5b358e297c|           52|                1.0|1592430500|
|00544ad7-df28-4b9...|5c5b358ef9b5|5c5b358f1620|   -65.0|          8|00544ad7-df28-4b9...|5c5b358f1620|           52|                1.0|1592334948|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|   -56.0|          8|0112b19a-f4d4-4f2...|5c5b352ec92c|          112|                3.0|1591074919|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|   -56.0|          8|0112b19a-f4d4-4f2...|5c5b352ec92c|   

In [17]:
# ap impacted by ap2's radar_channel
ap_neighbor_radar_3 = ap_neighbor_radar_2\
                    .filter('radar_channel>0')\
                    .select('site', 'ap', 'ap2', 'radar_channel', 'radar_channel_count', 'max_rssi')\
                     .groupBy('site', 'ap', 'ap2', 'radar_channel')\
                     .agg(F.max('max_rssi').alias("max_rssi"))
ap_neighbor_radar_3.show()

+--------------------+------------+------------+-------------+--------+
|                site|          ap|         ap2|radar_channel|max_rssi|
+--------------------+------------+------------+-------------+--------+
|0009dab3-7a6b-4ac...|5c5b358e2878|5c5b358e297c|           52|   -57.0|
|00544ad7-df28-4b9...|5c5b358ef9b5|5c5b358f1620|           52|   -65.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          100|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          104|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          108|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          112|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          116|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          120|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          124|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          128|   -56.0|
|0112b19a-f4d4-4f2...|5c5b352ec90e|5c5b352ec92c|          132|  

In [18]:
# aggregated impacted by ap2's radar_channel
ap_neighbor_radar_4 = ap_neighbor_radar_3.groupBy('site', 'ap',  'radar_channel')\
         .agg(F.collect_list('ap2').alias('neighbor_aps'), F.collect_list('max_rssi').alias('neighbor_rssis'))\
        .withColumnRenamed('radar_channel', 'neighbor_radar_channel')
ap_neighbor_radar_4.show()

+--------------------+------------+----------------------+--------------------+--------------------+
|                site|          ap|neighbor_radar_channel|        neighbor_aps|      neighbor_rssis|
+--------------------+------------+----------------------+--------------------+--------------------+
|0112b19a-f4d4-4f2...|5c5b350f1fd7|                   136|[5c5b352f06da, 5c...|[-74.0, -68.0, -7...|
|0129e71e-cf17-4ad...|5c5b35ce2c56|                    64|      [5c5b35bf55fe]|             [-64.0]|
|0129e71e-cf17-4ad...|5c5b35ce4a1f|                    56|      [5c5b35ce4c04]|             [-67.0]|
|01e2e0f1-1238-440...|5c5b358e2418|                    60|      [5c5b358e2413]|             [-53.0]|
|020ca1e5-b776-441...|5c5b35bed40d|                   128|      [5c5b35bed359]|             [-65.0]|
|0286e436-d197-45b...|5c5b35cea0d7|                   124|[5c5b35cea1a9, 5c...|      [-69.0, -66.0]|
|02fd5d4f-556e-4db...|5c5b354f008b|                   108|[5c5b354f02c0, 5c...|      [-70.0

In [19]:
# join for radar_last_seen_in_site
ap_neighbor_radar_5 = ap_neighbor_radar_4\
                .join(last_seen_in_site_df, [ap_neighbor_radar_4.site == last_seen_in_site_df.site_id, 
                 ap_neighbor_radar_4.neighbor_radar_channel==last_seen_in_site_df.radar_channel], 'inner')\
                .select('site', 'ap', 'radar_channel', 'neighbor_rssis', 'neighbor_aps', 'last_seen_in_site')

ap_neighbor_radar_5.show()

+--------------------+------------+-------------+--------------------+--------------------+-----------------+
|                site|          ap|radar_channel|      neighbor_rssis|        neighbor_aps|last_seen_in_site|
+--------------------+------------+-------------+--------------------+--------------------+-----------------+
|0112b19a-f4d4-4f2...|5c5b350f1fd7|          136|[-68.0, -74.0, -6...|[5c5b352ec90e, 5c...|       1591080620|
|0129e71e-cf17-4ad...|5c5b35ce2c56|           64|             [-64.0]|      [5c5b35bf55fe]|       1593544807|
|0129e71e-cf17-4ad...|5c5b35ce4a1f|           56|             [-67.0]|      [5c5b35ce4c04]|       1591869233|
|01e2e0f1-1238-440...|5c5b358e2418|           60|             [-53.0]|      [5c5b358e2413]|       1591033217|
|020ca1e5-b776-441...|5c5b35bed40d|          128|             [-65.0]|      [5c5b35bed359]|       1592297118|
|0286e436-d197-45b...|5c5b35cea0d7|          124|      [-66.0, -69.0]|[5c5b35cea159, 5c...|       1593502128|
|02fd5d4f-

In [20]:

scan_final_df = ap_neighbor_radar_5
scan_final_df.show()

+--------------------+------------+-------------+--------------------+--------------------+-----------------+
|                site|          ap|radar_channel|      neighbor_rssis|        neighbor_aps|last_seen_in_site|
+--------------------+------------+-------------+--------------------+--------------------+-----------------+
|0112b19a-f4d4-4f2...|5c5b350f1fd7|          136|[-67.0, -70.0, -6...|[5c5b352f06df, 5c...|       1591080620|
|0129e71e-cf17-4ad...|5c5b35ce2c56|           64|             [-64.0]|      [5c5b35bf55fe]|       1593544807|
|0129e71e-cf17-4ad...|5c5b35ce4a1f|           56|             [-67.0]|      [5c5b35ce4c04]|       1591869233|
|01e2e0f1-1238-440...|5c5b358e2418|           60|             [-53.0]|      [5c5b358e2413]|       1591033217|
|020ca1e5-b776-441...|5c5b35bed40d|          128|             [-65.0]|      [5c5b35bed359]|       1592297118|
|0286e436-d197-45b...|5c5b35cea0d7|          124|      [-66.0, -69.0]|[5c5b35cea159, 5c...|       1593502128|
|02fd5d4f-

In [21]:
# ap_neighbor_radar_22 = scan_df_2\
#          .filter('ap2 is not Null and max_rssi >-75')\
#          .join(rrm_radar_df, [scan_df_2.site == rrm_radar_df.site_id, scan_df_2.ap2 == rrm_radar_df.ap_id], 'inner')\
#          .select('site', 'ap', 'ap2', 'pre_channel', 'radar_channel_count', 'max_rssi')\
#          .groupBy('site', 'ap', 'ap2', 'pre_channel')\
#          .agg(F.max('max_rssi').alias("max_rssi"))\
#         .groupBy('site', 'ap', 'pre_channel')\
#          .agg(F.collect_list('ap2').alias('neighbor_aps'), F.collect_list('max_rssi').alias('neighbor_rssis'))\
#         .withColumnRenamed('pre_channel', 'neighbor_radar_channel')                                                                                          

# ap_neighbor_radar_22.show()  

In [22]:
# ap_neighbor_radar_2.select('ap', 'neighbor_radar_channel', 'neighbor_aps', 'neighbor_rssis').show(2, False)  

In [23]:

# Goal: 

#rrm/policy/org/{org}/site/{site}/ap/ { "116" : {"weight_tot": 0.8, "radar_detected":0.5, "neighbor_radar": 0.3, "geolocation": 0.0, "last_seen": 11111}}

In [24]:
scan_final_df_site = scan_final_df.filter(col('site')==site_1)
scan_final_df_site.show()


name 'col' is not defined
Traceback (most recent call last):
NameError: name 'col' is not defined



# channel weight

In [25]:
import numpy as np
def weight_by_dfs_channel(chn, wt_0=0.4, wt_weather_ch=0.1, count=1 ):
    if 116<=chn<132:
        wt = wt_0 + wt_weather_ch
    else:
        wt = wt_0
    
    wt = 1 - (1-wt)**count
    return wt

def weight_by_dist(dist_to_airport=100.0, max_wt=0.8):
    wt = 0.0
    if dist_to_airport< 10:
        wt = max_wt * np.exp(-dist_to_airport**2/5**2)
    
    return wt

def weight_impacted_neighbor(rssi, rssi_0 = -65):
    wt = 1.0
    if rssi < rssi_0:
        wt =  np.exp((rssi - rssi_0)/15) 
    return wt
    
    
def weight_tot(ch, rssi=-40, dist_to_airport=100.0, theta=[0.33, 0.33, 0.33]):
    wt=[0, 0, 0]
    wt[0] = weight_by_dfs_channel(ch)
    wt[1] = weight_by_dist(dist_to_airport)
    wt[2] = weight_impacted_neighbor(rssi)
    
    wt_0 = 0.0
    for i in range(2):
        wt_0 += wt[i] * theta[i]
    
#     print(wt_0, wt)
#     wt_0 = 1.0/(1.0 + np.exp(-0.6*wt_0))
    wt_0 = (wt[0] + wt[1]) * wt[2]

    return wt_0


In [26]:
rrm_radar_df.show(2)

+--------------------+------------+-------------+-------------------+---------------+----------+
|             site_id|       ap_id|radar_channel|radar_channel_count|last_radar_time|      time|
+--------------------+------------+-------------+-------------------+---------------+----------+
|ad1fe7ce-b78d-458...|5c5b352f5707|          132|                  1|     1592678555|1592676000|
|1cebf4b4-3b20-4db...|5c5b35bf2fd9|           60|                  1|     1592679386|1592676000|
+--------------------+------------+-------------+-------------------+---------------+----------+
only showing top 2 rows

In [27]:
# rrm_radar_df.printSchema()

In [28]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode

from pyspark.sql.types import IntegerType, FloatType


import time
t0 = int(time.time())

wt_0 = 0.4
wt_weather_ch = 0.2 
def weight_by_dfs_channel(chn, count=1 ):
    
    if 116<=int(chn)<132:
        wt = wt_0 + wt_weather_ch
    else:
        wt = wt_0
    
#     time_diff  = (t0 - int(last_radar_time))/3600./24
    wt = (1 - (1-wt)**int(count)) #* np.exp(-time_diff)
    

    return wt

primary_weight = udf(weight_by_dfs_channel, FloatType())
rrm_radar_wt_df = rrm_radar_df.withColumn("weight",
                                          primary_weight(col("radar_channel"),
                                                         col("radar_channel_count")
#                                                          col("last_radar_time")
                                                        )
                                         )
rrm_radar_wt_df.show()

+--------------------+------------+-------------+-------------------+---------------+----------+------+
|             site_id|       ap_id|radar_channel|radar_channel_count|last_radar_time|      time|weight|
+--------------------+------------+-------------+-------------------+---------------+----------+------+
|ad1fe7ce-b78d-458...|5c5b352f5707|          132|                  1|     1592678555|1592676000|   0.4|
|1cebf4b4-3b20-4db...|5c5b35bf2fd9|           60|                  1|     1592679386|1592676000|   0.4|
|13d68843-7ebd-42a...|d420b0008b8d|           52|                  1|     1592678522|1592676000|   0.4|
|5fe533f6-39f5-431...|5c5b35500622|          108|                  1|     1592677880|1592676000|   0.4|
|03ba030e-2a1f-4d5...|5c5b359ed877|          108|                  1|     1592678957|1592676000|   0.4|
|7e437c8b-65fe-45d...|5c5b35bf5c52|          132|                  1|     1592678927|1592676000|   0.4|
|1b48d79e-a581-41d...|5c5b352e519a|           60|               

In [29]:

# def weight_impacted_neighbor(chn, neighbor_rssis, rssi_0 = -65):
#     if 116<=int(chn)<132:
#         wt_ch = wt_0 + wt_weather_ch
#     else:
#         wt_ch = wt_0
    
#     wt = 1.0
#     for rssi_s in neighbor_rssis:
#         rssi = float(rssi_s)
#         if rssi < rssi_0:
#             wt +=  np.exp((rssi - rssi_0)/15) 
#         else:
#             wt = 1.0
    
# #     time_diff  = (t0 - int(last_radar_time))/3600./24
#     wt = wt* wt_ch  #* np.exp(-time_diff)
#     return wt*wt_ch
    
# neighbor_weight = udf(weight_impacted_neighbor, FloatType())

# scan_final_df.show()
# scan_final_wt_df = scan_final_df.withColumn("weight", neighbor_weight(col("radar_channel"), col("neighbor_rssis")))
# scan_final_wt_df.show()


In [30]:
rrm_radar_wt_df_site = rrm_radar_wt_df.filter(col("site_id")==site_1)
rrm_radar_wt_df_site.show()

+--------------------+------------+-------------+-------------------+---------------+----------+------+
|             site_id|       ap_id|radar_channel|radar_channel_count|last_radar_time|      time|weight|
+--------------------+------------+-------------+-------------------+---------------+----------+------+
|13d68843-7ebd-42a...|d420b0008b8d|           52|                  1|     1592678522|1592676000|   0.4|
|13d68843-7ebd-42a...|5c5b35bf2336|          128|                  1|     1592678373|1592676000|   0.6|
|13d68843-7ebd-42a...|d420b0008da9|          120|                  1|     1592678829|1592676000|   0.6|
|13d68843-7ebd-42a...|d420b00090a6|          104|                  1|     1592678438|1592676000|   0.4|
|13d68843-7ebd-42a...|d420b0008b33|          136|                  1|     1592678541|1592676000|   0.4|
|13d68843-7ebd-42a...|d420b000911e|           60|                  1|     1592678420|1592676000|   0.4|
|13d68843-7ebd-42a...|d420b0008ecb|           52|               

In [31]:
rrm_radar_wt_df_site\
    .select("ap_id", "radar_channel", "weight").orderBy("ap_id", "radar_channel")\
    .show(rrm_radar_wt_df_site.count(), False)

+------------+-------------+------+
|ap_id       |radar_channel|weight|
+------------+-------------+------+
|5c5b352f0b76|100          |0.4   |
|5c5b352f0b76|108          |0.4   |
|5c5b352f0b76|112          |0.4   |
|5c5b352f0b76|116          |0.6   |
|5c5b352f0b76|60           |0.4   |
|5c5b352f0b76|60           |0.4   |
|5c5b35bf21d3|120          |0.6   |
|5c5b35bf21d3|132          |0.4   |
|5c5b35bf220f|116          |0.6   |
|5c5b35bf220f|116          |0.6   |
|5c5b35bf220f|116          |0.6   |
|5c5b35bf2214|100          |0.4   |
|5c5b35bf2214|132          |0.4   |
|5c5b35bf2214|60           |0.4   |
|5c5b35bf2214|64           |0.4   |
|5c5b35bf223c|52           |0.4   |
|5c5b35bf225f|132          |0.4   |
|5c5b35bf2273|116          |0.6   |
|5c5b35bf2273|136          |0.4   |
|5c5b35bf2273|136          |0.4   |
|5c5b35bf2273|64           |0.4   |
|5c5b35bf2282|120          |0.6   |
|5c5b35bf2282|140          |0.4   |
|5c5b35bf2287|120          |0.6   |
|5c5b35bf2287|136          |

In [32]:
rrm_radar_wt_df_site.count()

369

In [33]:
site_1

'13d68843-7ebd-42a1-8528-495c41ccf8ef'

In [34]:
# rrm_radar_wt_df_site.select("ap").cou
rrm_radar_wt_df_site.select(F.countDistinct("ap_id")).show()

+---------------------+
|count(DISTINCT ap_id)|
+---------------------+
|                  105|
+---------------------+

In [1]:
rrm_radar_wt_df_site.select(F.countDistinct("ap_id")).show()

NameError: name 'rrm_radar_wt_df_site' is not defined

In [36]:
# redis 
# rrm/policy/site/{site}/ap/{ap}/channel  {"116" : 0.7, '124': 0.8, } 


In [37]:
%ls ~/

In [38]:
%pwd

'/home/wenfeng'

In [39]:
%ls /home/wenfeng